In [1]:
import numpy as np
import math
import random
import os
import torch
import scipy.spatial.distance 
from torch.utils.data import Dataset, DataLoader 
from torchvision import transforms, utils 
import torch.nn.functional as F
from time import time
import torch.nn as nn
import math
import copy





/home/parvez/.local/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/parvez/.local/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZNK3c1010TensorImpl36is_contiguous_nondefault_policy_implENS_12MemoryFormatE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
random.seed=42

In [3]:
path=os.path.join("/home/parvez","Dataset/ModelNet40_dataset/ModelNet40")
print(path)

/home/parvez/Dataset/ModelNet40_dataset/ModelNet40


In [4]:
folders=[dir for dir in sorted(os.listdir(path))]
classes={folder:i for i, folder in enumerate(folders)}



In [5]:
def read_off(file):
    off_header = file.readline().strip()
    if 'OFF' == off_header:
        n_verts, n_faces, __ = tuple([int(s) for s in file.readline().strip().split(' ')])
    else:
        n_verts, n_faces, __ = tuple([int(s) for s in off_header[3:].split(' ')])
    verts = [[float(s) for s in file.readline().strip().split(' ')] for i_vert in range(n_verts)]
    faces = [[int(s) for s in file.readline().strip().split(' ')][1:] for i_face in range(n_faces)]
    return verts, faces


In [6]:
with open(os.path.join(path, 'bed/train/bed_0001.off'),'r') as f:
    verts, faces=read_off(f)
      
   

In [7]:
i,j,k=np.array(faces).T
x,y,z=np.array(verts).T


In [8]:
class PointSampler(object):
    def __init__(self, output_size):
        assert isinstance(output_size,int)
        self.output_size=output_size
        
    def triangle_area(self, pt1, pt2, pt3):
        side_a=np.linalg.norm(pt1-pt2)
        side_b=np.linalg.norm(pt2-pt3)
        side_c=np.linalg.norm(pt3-pt1)
        s=0.5*(side_a+side_b+side_c)
        return max(s*(s-side_a)*(s-side_b)*(s-side_c),0)**0.5
    
    def sample_point(self, pt1,pt2,pt3):
        s,t=sorted([random.random(), random.random()])
        f=lambda i: s*pt1[i]+(t-s)*pt2[i]+(1-t)*pt3[i]
        return (f(0), f(1), f(2))
    
    def __call__(self, mesh):
        verts, faces=mesh
        verts=np.array(verts)
        areas=np.zeros((len(faces)))
        for i in range(len(areas)):
            areas[i]=(self.triangle_area(verts[faces[i][0]],
                                       verts[faces[i][1]],
                                       verts[faces[i][2]]))
            
        sampled_faces=(random.choices(faces,
                                     weights=areas,
                                     cum_weights=None,
                                     k=self.output_size))
        
        sampled_points=np.zeros((self.output_size, 3))
        for i in range(len(sampled_faces)):
            sampled_points[i]=(self.sample_point(verts[sampled_faces[i][0]],
                                                verts[sampled_faces[i][1]],
                                                verts[sampled_faces[i][2]]))
            
        return sampled_points
    
    

In [9]:
pointcloud=PointSampler(3000)((verts, faces))

In [10]:
print(pointcloud.shape)

(3000, 3)


In [11]:
class Normalize(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2
        
        norm_pointcloud=pointcloud -  np.mean(pointcloud, axis=0)
        norm_pointcloud /=np.max(np.linalg.norm(norm_pointcloud, axis=1))
        
        return norm_pointcloud
    


In [12]:
norm_pointcloud = Normalize()(pointcloud)


In [13]:
class ToTensor(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2
        return torch.from_numpy(pointcloud)
    

In [14]:
def default_transforms():
    return transforms.Compose([
        PointSampler(1024),
        Normalize(),
        ToTensor()
    ])



In [15]:
class PointCloudData(Dataset):
    def __init__(self,root_dir, valid=False, folder='train', transform=default_transforms()):
        self.root_dir=root_dir
        folders=[dir for dir in sorted(os.listdir(root_dir))]
        self.classes={folder:i for i, folder in enumerate(folders)}
        self.transforms=transform if not valid else default_transforms()
        self.valid=valid
        self.files=[]
        for category in self.classes.keys():
            new_dir=os.path.join(root_dir, category, folder)
            for file in os.listdir(new_dir):
                if file.endswith('.off'):
                    sample={}
                    sample['pcd_path']=os.path.join(new_dir, file)
                    sample['category']=category
                    self.files.append(sample)
                    
    
    def __len__(self):
        return len(self.files)
    
    
    def __preproc__(self, file):
        verts, faces=read_off(file)
        if self.transforms:
            pointcloud=self.transforms((verts, faces))
        
        return pointcloud 
    
    def __getitem__(self, idx):
        pcd_path=self.files[idx]['pcd_path']
        category=self.files[idx]['category']
        with open(pcd_path, 'r') as f:
            pointcloud=self.__preproc__(f)
        return {'pointcloud' : pointcloud,
                'category' : self.classes[category]}
    
    
    

In [16]:
train_ds=PointCloudData(path)
valid_ds=PointCloudData(path, valid=True, folder='test')


In [17]:
train_loader=DataLoader(dataset=train_ds, batch_size=32, shuffle=True)
valid_loader=DataLoader(dataset=valid_ds, batch_size=64)


In [18]:
config={'num_points':1024,
       'batch_size':11,
       'use_normals':False,
       'optimizer':'RangerVA',
       'lr':0.001,
       'decay_rate':1e-06,
       'epochs':500,
       'num_classes':40,
       'dropout':0.4,
       'M':4,
       'K':64,
       'd_m':512
       }


In [19]:
class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        
    def forward(self,input1, input2):
        _,_,D=input1.shape
        
        Q=input1
        K=input2
        V=input1
        
        attn_weights=torch.bmm(Q, K.transpose(1,2))/math.sqrt(D)
        
        feature=torch.bmm(attn_weights, V)
        
        return feature
    
      

In [20]:
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        
        
        self.fc1=nn.Linear(3,64)
        self.fc2=nn.Linear(64,128)
        self.fc3=nn.Linear(128,256)
        
        self.attention=Attention()
        
        self.fc4=nn.Linear(256,128)
        self.fc5=nn.Linear(128,64)
        self.fc6=nn.Linear(64,40)
        
        

        
        self.softmax=nn.Softmax(dim=2)
        
        
    def forward(self,input):
        input_ =input
       
        x=F.relu(self.fc1(input))
        x=F.relu(self.fc2(x))
        x=F.relu(self.fc3(x))
        
        x= self.attention(x,x)
        
        x=F.relu(self.fc4(x))
        x=F.relu(self.fc5(x))
        x=self.fc6(x)
        
        x=torch.max(x, 1)[0]
        x=torch.unsqueeze(x,1)
        
        
        output=self.softmax(x)
        
        return output 
  
    

In [21]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [22]:
model=Model()
model.to(device)


Model(
  (fc1): Linear(in_features=3, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=256, bias=True)
  (attention): Attention()
  (fc4): Linear(in_features=256, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=40, bias=True)
  (softmax): Softmax(dim=2)
)

In [23]:
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

criterion=nn.CrossEntropyLoss()


In [28]:
def train(model, train_loader,val_loader=None,epoch=1):
    for epoch in range(epoch):
        running_loss=0.0
        for i, data in enumerate(train_loader):
            inputs, labels=data['pointcloud'].to(device).float(), data['category'].to(device)
            optimizer.zero_grad()
            
            outputs=model(inputs)
            outputs=torch.squeeze(outputs)
            
            
            loss=criterion(outputs,labels)
            
            loss.backward()
            optimizer.step()
            
            running_loss+=loss.item()
            
            print("loss {} of batch {}: ".format(loss.item(),i))          

In [29]:
train(model,train_loader)

loss 3.668440103530884 of batch 0: 
loss 3.730940103530884 of batch 1: 
loss 3.637190103530884 of batch 2: 
loss 3.637190103530884 of batch 3: 
loss 3.637190103530884 of batch 4: 
loss 3.637190103530884 of batch 5: 
loss 3.699690103530884 of batch 6: 
loss 3.449690103530884 of batch 7: 
loss 3.574690103530884 of batch 8: 
loss 3.699690103530884 of batch 9: 
loss 3.574690103530884 of batch 10: 
loss 3.637190103530884 of batch 11: 
loss 3.637190103530884 of batch 12: 
loss 3.574690103530884 of batch 13: 
loss 3.543440103530884 of batch 14: 
loss 3.637190103530884 of batch 15: 
loss 3.668440103530884 of batch 16: 
loss 3.730940103530884 of batch 17: 
loss 3.668440103530884 of batch 18: 
loss 3.574690103530884 of batch 19: 
loss 3.480940103530884 of batch 20: 
loss 3.730940103530884 of batch 21: 
loss 3.730940103530884 of batch 22: 
loss 3.668440103530884 of batch 23: 
loss 3.574690103530884 of batch 24: 
loss 3.605940103530884 of batch 25: 
loss 3.605940103530884 of batch 26: 
loss 3.6684

KeyboardInterrupt: 

IndexError: The shape of the mask [32, 256, 3] at index 2 does not match the shape of the indexed tensor [32, 256, 16] at index 2

tensor([[[0.6794, 0.6302, 0.2393],
         [0.7705, 0.6448, 0.5285],
         [0.1703, 0.6312, 0.5203],
         [0.7421, 0.0197, 0.5319],
         [0.7266, 0.7388, 0.7423],
         [0.0330, 0.8191, 0.6201],
         [0.8280, 0.1459, 0.6869],
         [0.4378, 0.5661, 0.5191],
         [0.9522, 0.1850, 0.9113],
         [0.7879, 0.3846, 0.5760]],

        [[0.4122, 0.2750, 0.2543],
         [0.9969, 0.3531, 0.8177],
         [0.6538, 0.9346, 0.4969],
         [0.7928, 0.5726, 0.6629],
         [0.6816, 0.5527, 0.6366],
         [0.8280, 0.5558, 0.8445],
         [0.0375, 0.3176, 0.3728],
         [0.4986, 0.4665, 0.2881],
         [0.7322, 0.2450, 0.6097],
         [0.7538, 0.0550, 0.8272]],

        [[0.3140, 0.7076, 0.8817],
         [0.7900, 0.9898, 0.1162],
         [0.8440, 0.6232, 0.4816],
         [0.9902, 0.2445, 0.9466],
         [0.3084, 0.1920, 0.8627],
         [0.4768, 0.9245, 0.8775],
         [0.9604, 0.9177, 0.2915],
         [0.4730, 0.6404, 0.1862],
         [0.1363